# Internet Retrieval with Claude

This notebook provides a step-by-step guide for using the Brave search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query the Brave search engine
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [ ]:
import os
import sys
import dotenv
import anthropic
import nest_asyncio
nest_asyncio.apply()  # This is a workaround for asyncio + jupyter notebooks

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv()

In [ ]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

## Make a Brave Search Tool

After importing the BraveSearchTool class, we can easily initalize a new BraveSearchTool object.

Let's test the search tool to make sure it works:

In [ ]:
from claude_retriever.searcher.searchtools.websearch import BraveSearchTool

# Create a searcher
brave_search_tool = BraveSearchTool(brave_api_key=os.environ["BRAVE_API_KEY"], summarize_with_claude=True, anthropic_api_key=os.environ["ANTHROPIC_API_KEY"])

# Run a test query
query = "2023 World Cup"

results = brave_search_tool.search(query=query, n_search_results_to_use=3)
print(results)

## Use Claude with Retrieval

Now we simply pass the search tool to a Claude client, allowing Claude access to use it when generating responses. 

In [ ]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], search_tool = brave_search_tool)

query = "Who scored the most goals in the 2023 Women's Soccer World Cup?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [ ]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

Now let's try getting a completion from Claude equipped with the Brave search tool.

In [ ]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=1,
    max_searches_to_try=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)